In [1]:
import json
import pandas as pd
import numpy as np

# 데이터셋 로드
with open('../../dataset/train.json') as f:
    data = json.load(f)

# 데이터프레임으로 변환
df = pd.DataFrame(data['images'])

# 데이터의 20%를 랜덤으로 선택
val_indices = np.random.choice(df.index, size=int(len(df) * 0.1), replace=False)
val_df = df.loc[val_indices]
train_df = df.drop(val_indices)

# annotations 필터링
train_annotations = []
val_annotations = []

for annotation in data['annotations']:
    if annotation['image_id'] in val_df['id'].values:
        val_annotations.append(annotation)
    else:
        train_annotations.append(annotation)

# train_sep.json 및 val.json 파일로 저장
train_data = {
    'images': train_df.to_dict(orient='records'),
    'annotations': train_annotations,
    'categories': data['categories']  # categories는 그대로 유지
}
val_data = {
    'images': val_df.to_dict(orient='records'),
    'annotations': val_annotations,
    'categories': data['categories']  # categories는 그대로 유지
}

with open('../../dataset/train_sep.json', 'w') as f:
    json.dump(train_data, f, indent=4)

with open('../../dataset/val_sep.json', 'w') as f:
    json.dump(val_data, f, indent=4)

print("데이터셋을 train_sep.json과 val.json으로 나누었습니다.")


데이터셋을 train_sep.json과 val.json으로 나누었습니다.


In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.preprocessing import MultiLabelBinarizer
import json
import pandas as pd
import numpy as np

# 데이터셋 로드
with open('../../dataset/train.json') as f:
    data = json.load(f)

# 데이터프레임으로 변환
df = pd.DataFrame(data['images'])

# 이미지 id를 기준으로 annotation에서 레이블을 추출
image_labels = {image['id']: [] for image in data['images']}

# 각 이미지에 해당하는 레이블 추가
for annotation in data['annotations']:
    image_id = annotation['image_id']
    category_id = annotation['category_id']
    image_labels[image_id].append(category_id)

# 각 이미지의 멀티라벨 리스트를 데이터프레임에 추가
df['labels'] = df['id'].map(image_labels)

# MultiLabelBinarizer를 사용하여 멀티라벨을 이진 매트릭스로 변환
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['labels'])

# MultilabelStratifiedKFold 사용
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Split을 저장할 리스트 초기화
splits = []

# 각 fold에 대한 분할 수행
for train_idx, val_idx in mskf.split(df, y):
    train_df = df.iloc[train_idx]
    val_df = df.iloc[val_idx]

    # 각 split에 대해 annotations 필터링
    train_annotations = []
    val_annotations = []

    for annotation in data['annotations']:
        if annotation['image_id'] in val_df['id'].values:
            val_annotations.append(annotation)
        else:
            train_annotations.append(annotation)

    # train 및 validation 데이터를 저장
    train_data = {
        'images': train_df.to_dict(orient='records'),
        'annotations': train_annotations,
        'categories': data['categories']
    }
    val_data = {
        'images': val_df.to_dict(orient='records'),
        'annotations': val_annotations,
        'categories': data['categories']
    }
    
    # 각 fold에 대해 저장
    split_num = len(splits) + 1
    with open(f'../../dataset/train_split_{split_num}.json', 'w') as f:
        json.dump(train_data, f, indent=4)
    
    with open(f'../../dataset/val_split_{split_num}.json', 'w') as f:
        json.dump(val_data, f, indent=4)

    # 저장된 split 추가
    splits.append((train_idx, val_idx))

print(f'{len(splits)}개의 fold로 데이터가 분할되었습니다.')

In [ ]:
import json
import pandas as pd
from pycocotools.coco import COCO

# COCO JSON 파일 경로
coco_file = 'path/to/output.bbox.json'# 결과를 저장할 CSV 파일 경로
output_csv_file = 'path/to/pascal_voc_submission.csv'

# COCO 데이터셋 로드
coco = COCO(coco_file)
img_ids = coco.getImgIds()
class_num = 10  # 클래스 수

# 제출 양식 후처리
prediction_strings = []
file_names = []

# COCO 형식의 예측 결과 로드
with open(coco_file) as f:
    coco_results = json.load(f)

# COCO 결과에서 output 추출
output = coco_results['annotations']  # 'annotations' 키에서 바운딩 박스 정보 추출

# 예측 결과를 CSV 형식으로 변환
for i, img_id in enumerate(img_ids):
    prediction_string = ''
    image_info = coco.loadImgs(img_id)[0]  # 이미지 정보를 가져옴

    # 클래스별 예측된 bbox가 있는지 확인 후 처리
    for j in range(class_num):
        # COCO에서 클래스별 예측 결과 필터링
        filtered_results = [o for o in output if o['category_id'] == j and o['image_id'] == img_id]
        
        if len(filtered_results) > 0:
            for o in filtered_results:
                # bbox는 [x, y, width, height] 형식이므로 [x, y, x+w, y+h]로 변환
                bbox = o['bbox']
                prediction_string += f'{j} {o["score"]:.6f} {bbox[0]:.2f} {bbox[1]:.2f} {bbox[0] + bbox[2]:.2f} {bbox[1] + bbox[3]:.2f} '
    # 예측된 결과가 없으면 '0 0 0 0 0 0' 형식으로 채움
    if prediction_string == '':
        prediction_string = '0 0 0 0 0 0'
    
    prediction_strings.append(prediction_string.strip())  # 공백 제거
    file_names.append(image_info['file_name'])  # 파일 이름 추가

# 제출 데이터 프레임 생성
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names

# 제출 파일 저장
submission.to_csv(output_csv_file, index=None)

# 결과 확인
print(submission.head())

In [ ]:
import pandas as pd
import pickle

# 결과물 파일 경로
results_file = './results.pkl'  # 결과물 파일 경로
output_csv_file = './pascal_voc_submission.csv'  # 결과를 저장할 CSV 파일 경로

# 결과물 파일 로드
with open(results_file, 'rb') as f:
    results = pickle.load(f)

# 결과물에서 필요한 정보 추출
prediction_strings = []
file_names = []

# 결과물에서 예측 결과를 CSV 형식으로 변환
for result in results:
    prediction_string = ''
    image_info = result['image_info']  # 이미지 정보를 가져옴

    # 클래스별 예측된 bbox가 있는지 확인 후 처리
    for class_id, bboxes in result['bboxes'].items():
        for bbox in bboxes:
            # bbox는 [x, y, width, height] 형식이므로 [x, y, x+w, y+h]로 변환
            prediction_string += f'{class_id} {bbox["score"]:.6f} {bbox["bbox"][0]:.2f} {bbox["bbox"][1]:.2f} {bbox["bbox"][0] + bbox["bbox"][2]:.2f} {bbox["bbox"][1] + bbox["bbox"][3]:.2f} '
    # 예측된 결과가 없으면 '0 0 0 0 0 0' 형식으로 채움
    if prediction_string == '':
        prediction_string = '0 0 0 0 0 0'
    
    prediction_strings.append(prediction_string.strip())  # 공백 제거
    file_names.append(image_info['file_name'])  # 파일 이름 추가

# 제출 데이터 프레임 생성
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names

# 제출 파일 저장
submission.to_csv(output_csv_file, index=None)

# 결과 확인
print(submission.head())